* hparams search:

https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py


In [19]:
# !conda install pandas numpy scipy ipython -y
# !conda install -c maciejkula -c pytorch spotlight -y
# !pip install spotlight

# !conda install pytorch torchvision  cudatoolkit=10.2 -c pytorch --force-reinstall -y

In [20]:
import sys
# sys.path.append("../../")
import os
# import papermill as pm

import pandas as pd
import numpy as np
# import tensorflow as tf
# from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, get_top_k_items, auc

from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, GroupKFold
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from spotlight.cross_validation import random_train_test_split, user_based_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.interactions import Interactions
import torch
from spotlight.evaluation import sequence_mrr_score,precision_recall_score
# from spotlight.evaluation import sequence_precision_recall_score # https://maciejkula.github.io/spotlight/evaluation.html#spotlight.evaluation.sequence_precision_recall_score

In [21]:
# from spotlight_hyperparams_movielens_sequence import *

# NUM_SAMPLES = 50

# LEARNING_RATES = [1e-3, 1e-2, 5 * 1e-2, 1e-1]
# LOSSES = ['bpr', 'hinge', 'adaptive_hinge', 'pointwise']
# BATCH_SIZE = [16,64, 128, 512]
# EMBEDDING_DIM = [32, 64, 128, 256]
# # N_ITER = list(range(5, 20))
# N_ITER = [10,25]
# L2 =[1e-6, 1e-4,  5e-3,1e-2,1e-1, 0.0]  ### [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.0] 

random_state = np.random.RandomState(100)

# CUDA = (os.environ.get('CUDA') is not None or
#         shutil.which('nvidia-smi') is not None)
# print(CUDA)
CUDA = False ## pytorch issues

max_sequence_length = 12
min_sequence_length = 3
random_state = np.random.RandomState(100)

In [22]:
def evaluate_accuracy_at_4(submission,ground_truth):
    '''checks if the true city is within the four recommended cities'''
    data_to_eval = submission.join(ground_truth,on='utrip_id')
    hits = data_to_eval.apply(
        lambda row: row['city_id'] in (row[['city_id_1', 'city_id_2', 'city_id_3', 'city_id_4']].values),
            axis = 1)
    return hits.mean()    


def evaluate_lstm_model(hyperparameters, train, test,random_state):

    h = hyperparameters

    model = ImplicitSequenceModel(loss=h['loss'],
                                  representation='lstm',
                                  batch_size=h['batch_size'],
                                  learning_rate=h['learning_rate'],
                                  l2=h['l2'],
                                  n_iter=h['n_iter'],
                                  use_cuda=CUDA,
                                  random_state=random_state)

    model.fit(train, verbose=True)

    test_mrr = sequence_mrr_score(model, test)
    print("test mrr",round(test_mrr,4))
    #val_mrr = sequence_mrr_score(model, validation)
    test_mrr = sequence_precision_recall_score(model, test, k=1, exclude_preceding=False )[0]
    # test_rec_prec = sequence_precision_recall_score(model, test, k=1, exclude_preceding=True )[0]
    # val_rec_prec = sequence_precision_recall_score(model, validation, k=4, exclude_preceding=True )[0]

    return test_mrr, model  

def evaluate_pooling_model(hyperparameters, train, test, random_state):

    h = hyperparameters

    model = ImplicitSequenceModel(loss=h['loss'],
                                  representation='pooling',
                                  batch_size=h['batch_size'],
                                  learning_rate=h['learning_rate'],
                                  l2=h['l2'],
                                  n_iter=h['n_iter'],
                                  use_cuda=CUDA,
                                  random_state=random_state)

    model.fit(train, verbose=True)

    test_mrr = sequence_mrr_score(model, test)
    #val_mrr = sequence_mrr_score(model, validation)
    #test_rec_prec = sequence_precision_recall_score(model, test)[0]
    #val_rec_prec = sequence_precision_recall_score(model, validation)[0]
    
    return test_mrr, model

In [23]:
torch.cuda.is_available()

True

In [24]:
torch.version.cuda

'8.0'

In [25]:
def sequence_precision_recall_score(model, test, k=1, exclude_preceding=False):
    """
    https://maciejkula.github.io/spotlight/_modules/spotlight/evaluation.html#sequence_precision_recall_score
    
    Compute sequence precision and recall scores. Each sequence
    in test is split into two parts: the first part, containing
    all but the last k elements, is used to predict the last k
    elements.

    Parameters
    ----------

    model: fitted instance of a recommender model
        The model to evaluate.
    test: :class:`spotlight.interactions.SequenceInteractions`
        Test interactions.
    exclude_preceding: boolean, optional
        When true, items already present in the sequence will
        be excluded from evaluation.

    Returns
    -------

    mrr scores: numpy array of shape (num_users,)
        Array of MRR scores for each sequence in test.
    """
    sequences = test.sequences[:, :-k]
    targets = test.sequences[:, -k:]
    precision_recalls = []
    for i in range(len(sequences)):
        predictions = -model.predict(sequences[i])
        if exclude_preceding:
            predictions[sequences[i]] = FLOAT_MAX

        predictions = predictions.argsort()[:k]
        precision_recall = _get_precision_recall(predictions, targets[i], k)
        precision_recalls.append(precision_recall)

    precision = np.array(precision_recalls)[:, 0]
    recall = np.array(precision_recalls)[:, 1]
    return precision, recall


def _get_precision_recall(predictions, targets, k=4):

    predictions = predictions[:k]
    num_hit = len(set(predictions).intersection(set(targets)))

    return float(num_hit) / len(predictions), float(num_hit) / len(targets)


In [26]:
# top k items to recommend
TOP_K = 4

# # Model parameters
# EPOCHS = 10
# BATCH_SIZE =  256#1024

# SEED = 0  # Set None for non-deterministic results
# random_state = np.random.RandomState(100)
# user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
# item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

TRAIN_FILE_PATH = os.path.normpath(r"C:\Users\Dan Ofer\Desktop\Stuff\booking_wisdom\booking_train_set.csv") #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"


Spotlight relevant examples:

https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py

https://github.com/maciejkula/spotlight/issues/172
* Dataset creation/loading + get predictions (sequence)

Spotlight hyperparameter tuning:
https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py

In [27]:
# df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE) ## orig 
# print(df.dtypes)
# print(df.shape)
# ## expected columns: ['userID', 'itemID', 'rating', 'timestamp'] (last as float), others as number
# df.head()

In [28]:
# df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE) ## orig 
df = pd.read_csv(TRAIN_FILE_PATH,
#                      nrows=230_500,
                     parse_dates=["checkin"],infer_datetime_format=True,usecols=["utrip_id","city_id","checkin"])

user_encoder = LabelEncoder()

## drop consecutive elements (need to do within group, and to order, and what about updating checkout date?)
## 0 cases if we also use checkin or checkout. 
## The "last"  criteria does drop rows! Do we want or not want it???
## We also +- need to merge with total duration data

df["utrip_id"] = user_encoder.fit_transform(df["utrip_id"])
df = df.groupby("utrip_id").filter(lambda x: len(x) >= 4) # keep only trips with at least 4

#####
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)
print("max rows", df["total_rows"].describe())
df["last"] = (df["row_num"] ==df["total_rows"])#.astype(int)
###
df = df.loc[(df[["city_id","utrip_id","last"]].shift() != df[["city_id","utrip_id","last"]]).max(axis=1)]

# ##########
# ### temp, drop longer trips, for easier evaluation rule of thumb
# print(df.shape[0])
# df = df.loc[df["total_rows"]<=max_sequence_length+1]
# print(df.shape[0])
##########
df = df.groupby("utrip_id").filter(lambda x: len(x) >= 4) # keep only trips with at least 4


df = df[["utrip_id","city_id","checkin","last"]]
              
df.columns = ['userID', 'itemID',  'timestamp',"last"] ## no rating col

df.sort_values(["userID","timestamp"],ascending=True,inplace=True) ## reinforce sort order for within groups

df = df[['userID', 'itemID',  'timestamp',"last"]]
print("nunique\n",df.nunique())
print(df.shape)
df.head()

max rows count    1.164595e+06
mean     6.123632e+00
std      2.795051e+00
min      4.000000e+00
25%      4.000000e+00
50%      5.000000e+00
75%      7.000000e+00
max      4.800000e+01
Name: total_rows, dtype: float64
nunique
 userID       195341
itemID        39560
timestamp       425
last              2
dtype: int64
(1042324, 4)


,userID,itemID,timestamp,last
542546,0,8183,2016-08-13,False
542547,0,15626,2016-08-14,False
542548,0,60902,2016-08-16,False
542549,0,30628,2016-08-18,True
1061281,1,38677,2016-04-09,False


In [29]:
print("unique items (city_id)")
print(df['itemID'].nunique())
df['itemID'].value_counts().describe()

unique items (city_id)
39560


count    39560.000000
mean        26.347927
std        184.627494
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
max       9060.000000
Name: itemID, dtype: float64

In [30]:
# ### replace rare variables (under 2 occurrences) with "-1" dummy OR with their hotel country! (as negative mapped number)
city_ids_counts = df["itemID"].value_counts()
print("before uniques",df["itemID"].nunique())
city_ids_counts = city_ids_counts.to_dict()
df["itemID"] = df["itemID"].where(df["itemID"].map(city_ids_counts)>2, -1)

## run encoder on cities (due to negative and to ensure ok ordering for spotlight)
le_itm = LabelEncoder()
df["itemID"] = le_itm.fit_transform(df["itemID"])

df["itemID"] = df["itemID"]+1 ## add 1 due to 0 being reserved for 0 padding
print("max",df['itemID'].max())
print("unique items (city_id)")
print(df['itemID'].nunique())
print(df['itemID'].value_counts().describe())

before uniques 39560
max 21218
unique items (city_id)
21218
count    21218.000000
mean        49.124517
std        301.280348
min          3.000000
25%          4.000000
50%          8.000000
75%         20.000000
max      24519.000000
Name: itemID, dtype: float64


In [31]:
train_inds, test_inds = next(GroupShuffleSplit(test_size=.25, n_splits=2,random_state = 0).split(df,groups=df['userID']))

train= df.iloc[train_inds]
test = df.iloc[test_inds]

# train = pd.concat([df.iloc[train_inds],
#                   X_test.loc[X_test["last"]==False]]).drop(["last"],axis=1) # ORIG
train = df.copy() # new dan = "train + test"

num_users_all=int(df['userID'].max()+1)
num_items_all=int(df['itemID'].max()+1)

last_city = test.loc[test["last"]==True]["itemID"] ##pd.concat([X_test.loc[X_test["last"]==True]]).drop(["last"],axis=1)

test_eval = test.loc[test["last"]!=True] ## test without last row

In [32]:
##### spotlight drops sequences that are too long instead of truncating them ??? 
## So we truncate ourselves! 

### Possible improvement: keep head (1) + tail  (i.e leaves info on first place visited

train = train.groupby("userID").tail(max_sequence_length)
# test = test.groupby("userID").tail(max_sequence_length)
test_eval= test_eval.groupby("userID").tail(max_sequence_length)


In [33]:
assert test_eval["userID"].nunique() == test["userID"].nunique()

* Create spotlight Interactions dataset
    * https://github.com/maciejkula/spotlight/issues/172
    * Could provide WEIGHTS (more weight to final row in sequence / last) 
    
    * Getting predictions: 
    ```
    predictions = model.predict(ids)

    item_ids= (-predictions).argsort()[:10] # last 10 items
    print(item_ids)
    print(predictions[item_ids])
    ```

In [34]:
print("max user",train['userID'].max())
print("max itemID",train['itemID'].max())

max user 217685
max itemID 21218


In [35]:
train = Interactions(user_ids=train['userID'].values,
                           item_ids=train['itemID'].values,
                           timestamps=train['timestamp'].values
                         ,num_users=num_users_all,num_items=num_items_all)
print(train)
test = Interactions(user_ids=test['userID'].values,
                           item_ids=test['itemID'].values,
                           timestamps=test['timestamp'].values
                    ,num_users=num_users_all,num_items=num_items_all)
print(test)

test_eval = Interactions(user_ids=test_eval['userID'].values,
                           item_ids=test_eval['itemID'].values,
                           timestamps=test_eval['timestamp'].values
                         ,num_users=num_users_all,num_items=num_items_all)


print(test_eval)

<Interactions dataset (217686 users x 21219 items x 1035934 interactions)>
<Interactions dataset (217686 users x 21219 items x 260975 interactions)>
<Interactions dataset (217686 users x 21219 items x 210953 interactions)>


In [36]:
# %%time
train_sequential = train.to_sequence(max_sequence_length, min_sequence_length=3)
print("train_sequential",train_sequential)
test_sequential = test.to_sequence(max_sequence_length, min_sequence_length=2)
print("test_sequential",test_sequential)

test_eval_sequential = test_eval.to_sequence(max_sequence_length, min_sequence_length=3)
print("test_eval_sequential",test_eval_sequential)

train_sequential <Sequence interactions dataset (195341 sequences x 12 sequence length)>
test_sequential <Sequence interactions dataset (49208 sequences x 12 sequence length)>
test_eval_sequential <Sequence interactions dataset (48836 sequences x 12 sequence length)>


In [37]:
hp = {'n_iter': 18, 'loss': 'hinge', 'learning_rate': 0.01, 'l2': 0.0, 'embedding_dim': 128, 'batch_size': 64}
# Best lstm result: {'test_mrr': 0.2304, 'validation_mrr': 0.256,
# hp_lstm = {'n_iter': 22, 'loss': 'adaptive_hinge', 'learning_rate': 0.008, 'l2': 0.0000001, 'embedding_dim': 128, 'batch_size': 128}

hp_lstm =  {'n_iter': 40, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 1e-09, 'embedding_dim': 128, 'batch_size': 128}
#  in hparam search  : Test MRR 0.2826 val MRR 0.2798 

In [20]:
%%time
test_mrr, model = evaluate_lstm_model(hp_lstm, train_sequential, test_sequential,random_state)
print(test_mrr.mean())

# pr, rec = sequence_precision_recall_score(model, test_sequential, k=1, exclude_preceding=False)
# print("prec",pr.mean())
# print("recall",rec.mean())

# Epoch 21: loss 0.14362004007497642
# 0.250 MRR


# ### new params, 40 epochs;
## hp_lstm =  {'n_iter': 40, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 1e-09, 'embedding_dim': 128, 'batch_size': 128}
# Epoch 38: loss 0.10279
# Epoch 39: loss 0.10273
# 0.2717458582912555
# # Wall time: 50min 22s

# Epoch 0: loss 0.4961106244769843
# Epoch 1: loss 0.33012199064212916
# Epoch 2: loss 0.22770608632722242
# Epoch 3: loss 0.17974120025238183
# Epoch 4: loss 0.15748407724895896
# Epoch 5: loss 0.14457095095298614
# Epoch 6: loss 0.13647255513234005
# Epoch 7: loss 0.13086477649528058
# Epoch 8: loss 0.12750605344479585
# Epoch 9: loss 0.1236967424317443
# Epoch 10: loss 0.12174199121932852
# Epoch 11: loss 0.12013898980172538
# Epoch 12: loss 0.11789007339023966
# Epoch 13: loss 0.1165304221578806
# Epoch 14: loss 0.11529011144189953
# Epoch 15: loss 0.11383789671876497
# Epoch 16: loss 0.11306551845345375
# Epoch 17: loss 0.11249355958151365
# Epoch 18: loss 0.11152296460623357
# Epoch 19: loss 0.11066758837343511
# Epoch 20: loss 0.10978671324335053
# Epoch 21: loss 0.10913099765231114
# Epoch 22: loss 0.10804773529487727
# Epoch 23: loss 0.10804745734967511
# Epoch 24: loss 0.10748752131911767
# Epoch 25: loss 0.10699583461706984
# Epoch 26: loss 0.10653344118515104
# Epoch 27: loss 0.10634881131394908
# Epoch 28: loss 0.10662320020078597
# Epoch 29: loss 0.10546379301699456
# Epoch 30: loss 0.10496348236792995
# Epoch 31: loss 0.10467267999841484
# Epoch 32: loss 0.10452803004028052
# Epoch 33: loss 0.10408895642306966
# Epoch 34: loss 0.10391999891402288
# Epoch 35: loss 0.10370281033252061
# Epoch 36: loss 0.10311332009074027
# Epoch 37: loss 0.10324403789984922
# Epoch 38: loss 0.10279558879306837
# Epoch 39: loss 0.10273740095889639
# 0.2717458582912555
# Wall time: 50min 22s

Epoch 0: loss 0.4961106244769843
Epoch 1: loss 0.33012199064212916
Epoch 2: loss 0.22770608632722242
Epoch 3: loss 0.17974120025238183
Epoch 4: loss 0.15748407724895896
Epoch 5: loss 0.14457095095298614
Epoch 6: loss 0.13647255513234005
Epoch 7: loss 0.13086477649528058
Epoch 8: loss 0.12750605344479585
Epoch 9: loss 0.1236967424317443
Epoch 10: loss 0.12174199121932852
Epoch 11: loss 0.12013898980172538
Epoch 12: loss 0.11789007339023966
Epoch 13: loss 0.1165304221578806
Epoch 14: loss 0.11529011144189953
Epoch 15: loss 0.11383789671876497
Epoch 16: loss 0.11306551845345375
Epoch 17: loss 0.11249355958151365
Epoch 18: loss 0.11152296460623357
Epoch 19: loss 0.11066758837343511
Epoch 20: loss 0.10978671324335053
Epoch 21: loss 0.10913099765231114
Epoch 22: loss 0.10804773529487727
Epoch 23: loss 0.10804745734967511
Epoch 24: loss 0.10748752131911767
Epoch 25: loss 0.10699583461706984
Epoch 26: loss 0.10653344118515104
Epoch 27: loss 0.10634881131394908
Epoch 28: loss 0.1066232002007859

In [21]:
# %%time
# test_mrr_pool, model_pool = evaluate_pooling_model(hp, train_sequential, test_sequential, random_state)
# print(test_mrr_pool.mean())

# pr, rec = sequence_precision_recall_score(model_pool, test_sequential, k=1, exclude_preceding=False)
# print("prec",pr.mean())
# print("recall",rec.mean())

In [22]:
%%time

preds = []
for i in range(len(test_eval_sequential.sequences)): 
    if (i //50) == int: print (i)
    predictions = model.predict(test_eval_sequential.sequences[i])
    item_ids= (-predictions).argsort()[:TOP_K+1]  ## k+1 (in case we have a negative/UNK to drop)
    item_ids = list(le_itm.inverse_transform(item_ids-1))
    ### drop predictions that are "unknown cities" (-1), and kee
    item_ids = [i for i in item_ids if i>= 0] ## drop negatives/ -1s
    item_ids = item_ids[:TOP_K]
    
    preds.append(item_ids)
print(len(preds))

df_preds = pd.DataFrame(data={"city_id": le_itm.inverse_transform(last_city-1),
                             "city_id_preds":preds})

print(df_preds)

print("top k accuracy @",TOP_K)
print(df_preds.apply(
        lambda row: row['city_id'] in (row["city_id_preds"]),
            axis = 1).mean())


# top k accuracy @ 4 : 0.361
# top k accuracy @ 50 0.779

48836
       city_id                 city_id_preds
0        42503   [6788, 24910, 40250, 52177]
1        28319  [30688, 17775, 10060, 18508]
2        52815  [52815, 21578, 28051, 18033]
3        64824  [12308, 64824, 27624, 51135]
4         4932   [4932, 17013, 60222, 50957]
...        ...                           ...
48831    29770  [46258, 29770, 55196, 41781]
48832    52571  [45454, 61320, 52571, 40069]
48833     6582  [56775, 12708, 34342, 56184]
48834    47700  [55763, 25946, 61320, 20967]
48835    20345  [62541, 28829, 44489, 58819]

[48836 rows x 2 columns]
top k accuracy @ 4
0.3612498976165124
Wall time: 5min 37s


In [23]:
# %%time
# pr, rec = sequence_precision_recall_score(model, test_sequential, k=1, exclude_preceding=False)
# print("prec",pr.mean()) ## 0.1427
# print("recall",rec.mean()) #

prec 0.14276353509705955
recall 0.14276353509705955
Wall time: 5min 15s


In [24]:
preds = []
for i in range(len(test_eval_sequential.sequences)): 
    predictions = model.predict(test_eval_sequential.sequences[i])
    item_ids= (-predictions).argsort()[:50+1]  ## k+1 (in case we have a negative/UNK to drop)
    item_ids = list(le_itm.inverse_transform(item_ids-1))
    ### drop predictions that are "unknown cities" (-1), and kee
    item_ids = [i for i in item_ids if i>= 0] ## drop negatives/ -1s
    item_ids = item_ids[:50]
    
    preds.append(item_ids)
print(len(preds))

df_preds = pd.DataFrame(data={"city_id": le_itm.inverse_transform(last_city-1),
                             "city_id_preds":preds})

print(df_preds)

print("top k accuracy 50")
print(df_preds.apply(
        lambda row: row['city_id'] in (row["city_id_preds"]),
            axis = 1).mean())
## @50: 0.779

48836
       city_id                                      city_id_preds
0        42503  [6788, 24910, 40250, 52177, 17127, 61187, 1419...
1        28319  [30688, 17775, 10060, 18508, 38382, 1745, 4790...
2        52815  [52815, 21578, 28051, 18033, 7456, 62399, 6988...
3        64824  [12308, 64824, 27624, 51135, 343, 60282, 29562...
4         4932  [4932, 17013, 60222, 50957, 38912, 26007, 9680...
...        ...                                                ...
48831    29770  [46258, 29770, 55196, 41781, 53363, 2201, 3760...
48832    52571  [45454, 61320, 52571, 40069, 27558, 834, 22696...
48833     6582  [56775, 12708, 34342, 56184, 6582, 1528, 53793...
48834    47700  [55763, 25946, 61320, 20967, 36063, 18850, 969...
48835    20345  [62541, 28829, 44489, 58819, 22490, 42482, 203...

[48836 rows x 2 columns]
top k accuracy @ 4
0.7797731181914981


In [25]:
# %%time
#### pool model : top k accuracy @ 4 :  0.246   (on 20% data)
# preds = []
# for i in range(len(test_eval_sequential.sequences)): 
#     predictions = model_pool.predict(test_eval_sequential.sequences[i])
#     item_ids= (-predictions).argsort()[:TOP_K+1]  ## k+1 (in case we have a negative/UNK to drop)
#     item_ids = list(le_itm.inverse_transform(item_ids-1))
#     ### drop predictions that are "unknown cities" (-1), and kee
#     item_ids = [i for i in item_ids if i>= 0] ## drop negatives/ -1s
#     item_ids = item_ids[:TOP_K]
    
#     preds.append(item_ids)
# print(len(preds))

# df_preds = pd.DataFrame(data={"city_id": le_itm.inverse_transform(last_city-1),
#                              "city_id_preds":preds})

# print(df_preds)

# print("top k accuracy @",TOP_K)
# print(df_preds.apply(
#         lambda row: row['city_id'] in (row["city_id_preds"]),
#             axis = 1).mean())

In [26]:
%%time
# run(train, test, validation, random_state, "lstm")

# Best lstm result: {'test_mrr': 0.2304, 'validation_mrr': 0.256,
# 'n_iter': 5, 'loss': 'adaptive_hinge', 'learning_rate': 0.05, 'l2': 0.0001, 'embedding_dim': 64, 'batch_size': 512}


# Evaluating {'n_iter': 20, 'loss': 'adaptive_hinge', 'learning_rate': 0.01, 'l2': 0.0001, 'embedding_dim': 128, 'batch_size': 128}
# Test MRR 0.269 val MRR 0.2738

Wall time: 0 ns


In [27]:
%%time
# run(train, test, validation, random_state, model_type="pooling")

# # Best pooling result: {'test_mrr': 0.28493, 'validation_mrr': 0.222,
# # 'n_iter': 15, 'loss': 'hinge', 'learning_rate': 0.01, 'l2': 0.0, 'embedding_dim': 64, 'batch_size': 16}

# Best pooling result: {'test_mrr': 0.2849, 'validation_mrr': 0.22,
#                       'n_iter': 15, 'loss': 'hinge', 'learning_rate': 0.01, 'l2': 0.0, 'embedding_dim': 64, 'batch_size': 16}

Wall time: 0 ns


In [28]:
# precision_recall_score(implicit_sequence_model, test, k=4)

In [29]:
# ## https://maciejkula.github.io/spotlight/evaluation.html#spotlight.evaluation.sequence_precision_recall_score
# ## listed in spotlight docs, not found on import ? 
# # Compute sequence precision and recall scores. 
# # Each sequence in test is split into two parts: the first part, containing all but the last k elements, is used to predict the last k elements.


# ### returns score per user
# seq_pr_score_array = sequence_precision_recall_score(implicit_sequence_model, test_sequential, k=1, exclude_preceding=False) 

# print("pr_score_array[0].mean",seq_pr_score_array[0].mean())
# print("pr_score_array[1].mean",seq_pr_score_array[1].mean()) ## the 2 arrays appear identical  ? 
# seq_pr_score_array